<a href="https://colab.research.google.com/github/ajraymond27/entertAIn/blob/master/Fast_Style_NST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fast NeuralStyleTransfer Model**
---
This notebook is for D.ai TRIP MEDIA use only. If you are accessing this notebook and wish to try it out, please make a copy in your personal Drive to work from. Thanks! 
---
---
About this notebook:
---
---
Neural style transfer is an optimization technique used to take two images—a content image and a style reference image (such as an artwork by a famous painter)—and blend them together so the output image looks like the content image, but “painted” in the style of the style reference image.

This is implemented by optimizing the output image to match the content statistics of the content image and the style statistics of the style reference image. These statistics are extracted from the images using a convolutional network.

For example, let’s take an image of this dog and Wassily Kandinsky's Composition 7:

<img src="https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg" width="500px"/>

<img src="https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg" width="500px"/>

Now how would it look like if Kandinsky decided to paint the picture of this Dog exclusively with this style? Something like this?

<img src="https://tensorflow.org/tutorials/generative/images/stylized-image.png" style="width: 500px;"/>


---
How to use this notebook:
---
---
For most AI models, the trickiest part is shaping the data in the exact form required by the model itself. You can't fit a square peg through a round hole. Here is a step by step guide to style your video content: 

**Step 1:** Select a video and style image
- Video will require preprocessing (see step 2)
- Save style image in PNG format

**Step 2:** Preprocess Video
- Import video to Adobe Premiere Pro (or other video editing service)
- Export video as a PNG image sequence 

**Step 3:** Save your input content in a folder structured like this: 
- example (you can name this folder anything you want. just keep it consistent)

> - content

>> - img_000
>> - img_001
>> - img_002
>> - img_003
>> - ... and all other images pulled from your video

> - style

>> - style.png

**Step 4:** Zip / Compress 'example' folder and upload to your Google Drive
- 'example.zip'
- Must be in MyDrive (not shared folder)

**Step 5:** Create a folder in the Fast NST --> Output folder with the same name you've been using 
- 'example'

**Step 6:** In this notebook, enable GPU by clicking Runtime --> Change runtime type --> Hardware accelerator = GPU; Runtime shape = HighRAM --> Save

- (NOTE: This model requires [colab pro](https://colab.research.google.com/signup) to process most video-length content. Regardless, it's suggested you process your image sequence in batches as described in step 6)

**Step 7:** Look forward in this notebook to update the name of your .zip file and set your batch size as explained in the code blocks
- (NOTE: With colab pro, you should be able to style several hundred images in each batch. Higher the resolution, lower the batch size. If your batch size is too large, you will see an error message stating you exceeded your RAM limit. Without colab pro, you wil only be able to process a few at a time) 

**Step 8:** Select Runtime --> Run all
- Watch each code block below as your image is styled through the model
- The last block will save your styled images to the Output --> example folder you created earlier

**Step 9:** Continue styling your images in batches by updating your batch settings
- After you changes your batch size settings, press play on the code block along with all the ones beneath
- Continue this step until all images are styled

**Step 10:** Download your Output --> example folder from Google Drive and complete your video editing process in your preferred video editing service

---
Start Creating! 
---
---
As you use this notebook, keep giving AJ feedback & ideas on how to make this process easier

# **Enabling and Testing GPU**

In [ ]:
#@title Hit This GPU Bruh
#Check if GPU is connected
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# How much GPU, tho?
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# What about RAM?
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# **Import & Configure Libraries & Models**

In [ ]:
#@title Art Vandalay, Importer / Exporter
import os
from os.path import exists
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
tf.__version__

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools
import time

# **Set Path to Content and Style Images**

In [ ]:
#@title Download Google
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

In [ ]:
#@title Authenticate Yourself
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#@title Search for your .zip file in Google Drive folder
#@markdown ---
#@markdown Format the two fields like this (copy/paste):

#@markdown query: title = 'example.zip'

#@markdown zip: example.zip

#@markdown ---

query = "title='example.zip'" #@param {type:"string"}
zip = "example.zip" #@param {type:"string"}

fid = drive.ListFile({'q': query}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile(zip)

In [ ]:
#@title Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# **Load Content and Style Images**

## Unzip your .zip
---
Change "example.zip" to your zip file name. The code below should look like this:

f.keys()

!unzip example.zip

---

In [ ]:
f.keys()
!unzip example.zip

In [ ]:
#@title Set Batch Size
#@markdown ---
first_image =  0#@param {type:"integer"}
last_image =  100#@param {type:"integer"}


In [ ]:
#@title Enter your desired output resolution
#@markdown ---
#@markdown Value should be width if in landscape orientation

#@markdown Value should be hight if in portrait orientation

#@markdown Example: 1920x1080px ... Value = 1920

#@markdown ---

max_dimension =  1920#@param {type:"integer"}

In [ ]:
#@title Download all content and style images

PATH = '/content/' + zip.strip(".zip")

if exists(os.path.join(PATH,
                          'content/.DS_Store')): 
  os.remove(os.path.join(PATH,
                          'content/.DS_Store'))

content_path = os.path.join(PATH,
                         'content/')

style_path = os.path.join(PATH,
                        'style/style.png')

def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

def load_img(path_to_img):
  max_dim = max_dimension
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

content_images = []
image_paths = sorted(os.listdir(content_path))

# iterate through the names of contents of the folder
for image_path in image_paths[first_image:last_image]:

  # create the full input path and read the file
  input_path = os.path.join(content_path, image_path)
  content_image = load_img(input_path)

  content_images.append(content_image)


style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_images[0], 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

# Fast Style Transfer

In [ ]:
#@title Run Fast Style Transfer
import tensorflow_hub as hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

fast_styled_images = []
start = time.time()

for i in content_images:
  stylized_image = hub_model(tf.constant(i), tf.constant(style_image))[0]
  tensor_to_image(stylized_image)
  fast_styled_images.append(stylized_image)

end = time.time()
print("Total time: {:.1f}".format(end-start))
print("Number of Styled Frames: {}".format(len(fast_styled_images)))
print("Example:")
# tensor_to_image(fast_styled_images[0])

# Save Styled Images

In [ ]:
#@title Save output to company drive folder
import os
os.chdir('/content/drive/Shareddrives/D.ai TRIP MEDIA/D.ai MODELS/Production Models/FAST_STYLE_NST/Output/' + zip.strip(".zip"))
!pwd

count = first_image

for i in fast_styled_images:
  file_name = 'fast-styled-image-' + str(count) + ".png"
  tensor_to_image(i).save(file_name)
  files.download(file_name)
  count += 1